# Standard Form conversion

Convert the following linear program into standard form:

$\min p+q$ subject to

$5p-3q=7$, $2p+q \ge 2$, $1 \le q \le 4$.

### Original problem

In [4]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)
@variable(m, p)
@variable(m, 1 <= q <= 4 )
@constraint(m, 5p - 3q == 7 )
@constraint(m, 2p + q >= 2 )
@objective(m, Min, p + q )

optimize!(m)

println(m)
println("p = ", value(p) )
println("q = ", value(q) )
println("objective = ", objective_value(m) )

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-2); columns 0(-2); elements 0(-4) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  3.0000000000e+00
HiGHS run time      :          0.00
Min p + q
Subject to
 5 p - 3 q = 7.0
 2 p + q ≥ 2.0
 q ≥ 1.0
 q ≤ 4.0

p = 2.0
q = 1.0
objective = 3.0


### Standard form
should look like:
$$\begin{aligned}
\text{maximize}\qquad& c^T x\\
\text{subject to:}\qquad& Ax \le b\\
& x \ge 0
\end{aligned}$$

In [5]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)
@variable(m, u >= 0 )
@variable(m, v >= 0 )
@variable(m, w >= 0 )
@constraint(m, (w+1) <= 4 )
@constraint(m, -5(u-v) + 3(w+1) <= -7 )
@constraint(m, 5(u-v) - 3(w+1) <= 7 )
@constraint(m, -2(u-v) - (w+1) <= -2 )
@objective(m, Max, -(u-v) - (w+1) )
println(m)

optimize!(m)
status= termination_status(m)

println(status)
println()
println("p = ", value(u-v) )
println("q = ", value(w+1) )
println("objective = ", -objective_value(m) )

Max -u + v - w - 1
Subject to
 w ≤ 3.0
 -5 u + 5 v + 3 w ≤ -10.0
 5 u - 5 v - 3 w ≤ 10.0
 -2 u + 2 v - w ≤ -1.0
 u ≥ 0.0
 v ≥ 0.0
 w ≥ 0.0

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
3 rows, 3 cols, 9 nonzeros
2 rows, 2 cols, 4 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-4); columns 0(-3); elements 0(-10) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     : -3.0000000000e+00
HiGHS run time      :          0.00
OPTIMAL

p = 2.0
q = 1.0
objective = 3.0


### Standard form (compact)

In [6]:
# we defined the vector x to be: x = [u;  v;  w]
A = [0 0 1; -5 5 3; 5 -5 -3; -2 2 -1]
b = [3; -10; 10; -1]
c = [-1; 1; -1]

using JuMP, HiGHS, LinearAlgebra  # load LinearAlgebra so we can use dot()

m = Model(HiGHS.Optimizer)
@variable(m, x[1:3] >= 0 )      # specify a vector variable
@constraint(m, A*x .<= b )      # the dot in front of <=, which indicates element-wise (vector) inequalities
@objective(m, Max, dot(c,x) )   # must use dot(c,x) or (c'*x)[1] to return a scalar
println(m)

optimize!(m)
status=termination_status(m)

println("terminated with status ",status)
println()
println("p = ", value(x[1]-x[2]) )
println("q = ", value(x[3]+1) )
println("objective = ", -(objective_value(m)-1) )

Max -x[1] + x[2] - x[3]
Subject to
 x[3] ≤ 3.0
 -5 x[1] + 5 x[2] + 3 x[3] ≤ -10.0
 5 x[1] - 5 x[2] - 3 x[3] ≤ 10.0
 -2 x[1] + 2 x[2] - x[3] ≤ -1.0
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
3 rows, 3 cols, 9 nonzeros
2 rows, 2 cols, 4 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-4); columns 0(-3); elements 0(-10) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     : -2.0000000000e+00
HiGHS run time      :          0.00
terminated with status OPTIMAL

p = 2.0
q = 1.0
objective = 3.0
